In [ ]:
import GPy, scipy, patsy
from gpmultipy import dataset

In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
ds = dataset.DataSet('data/')
x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',effects=['Well'])
y = np.log2(y)
y = (y-y.mean())/y.std()

In [ ]:
edge = range(101,111) + range(111,191,10) + range(120,191,10) + range(191,201) + range(201,211) + range(211,291,10) + range(220,291,10) + range(291,301)
edge = np.array(edge)
edge

In [ ]:
dist = np.array([min(abs(l-edge)) for l in labels[0]])
position = (dist==0).astype(int)

In [ ]:
xgp = np.zeros((x.shape[0]*y.shape[1], 2))
xgp[:,0] = np.tile(x[:,0], y.shape[1])
xgp[:,1] = np.repeat(position, x.shape[0])

ygp = y.ravel(1)[:,None]

xgp = xgp[::10]
ygp = ygp[::10]

xgp.shape

In [ ]:
# k = GPy.kern.RBF(2, ARD=True)
k = GPy.kern.RBF(1,name='time')*GPy.kern.RBF(1,name='edge',active_dims=[1])

m = GPy.models.GPRegression(xgp,ygp,k)
m

In [ ]:
m.optimize()
m

In [ ]:
k

In [ ]:
m.input_sensitivity()

In [ ]:
xpred = np.zeros((100,2))
xpred[:50,0] = xpred[50:,0] = np.linspace(x.min(),x.max())
xpred[50:,1] = 1

mu,cov = m.predict_noiseless(xpred,full_cov=True)

op = np.zeros((50,100))
for i in range(50):
    op[i,i] = 1
    op[i,i+50] = -1
    
mu = np.dot(op,mu)[:,0]
cov = np.dot(op,np.dot(cov,op.T))

plt.plot(xpred[:50,0],mu)
plt.fill_between(xpred[:50,0],mu+1.96*np.sqrt(cov.diagonal()),mu-1.96*np.sqrt(cov.diagonal()),alpha=.1)
plt.plot([x.min(),x.max()],[0,0],'k',lw=3)

plt.plot(x,y[:,position==0].mean(1) - y[:,position==1].mean(1))
plt.title('Non-edge vs edge',fontsize=20)
plt.xlabel("time", fontsize=15)
plt.ylabel("log(OD)", fontsize=15)
plt.tight_layout()

plt.savefig("figures/edge-effect-gp.pdf")

In [ ]:
cov = k.K(xpred)
cov = np.dot(op,np.dot(cov,op.T))

np.sqrt(cov.diagonal()*1.96)

In [ ]:
xpred = np.zeros((50,2))
xpred[:,0] = np.linspace(x.min(),x.max())
# xpred[50:,1] = 1

mu,cov = m.predict_noiseless(xpred,full_cov=True,kern=k.time)
mu = mu[:,0]

plt.plot(xpred[:50,0],mu)
plt.fill_between(xpred[:50,0],mu+1.96*np.sqrt(cov.diagonal()),mu-1.96*np.sqrt(cov.diagonal()),alpha=.1)
plt.plot([x.min(),x.max()],[0,0],'k',lw=3)